In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import ast
import itertools

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [3]:
principal_cells = pd.read_pickle('../data/rodent_3d_dendrites_br-ct-filter-3_principal_mainclasses.pkl')
interneurons = pd.read_pickle('../data/rodent_3d_dendrites_br-ct-filter-3_interneuron_mainclasses.pkl')
print(len(principal_cells), len(interneurons))

16146 1614


In [4]:
principal_cells.shape

(16146, 67)

In [5]:
interneurons.shape

(1614, 76)

In [6]:
interneurons.head()

,age_classification,archive,attributes,brain_region,cell_type,deposition_date,domain,experiment_condition,magnification,max_age,max_weight,min_age,min_weight,neuron_id,note,objective_type,original_format,physical_Integrity,protocol,reconstruction_software,reference_pmid,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_surface,species,stain,strain,upload_date,3D,angles,bif_ampl_local,branch_Order,contraction,depth,diameter,eucDistance,fractal_Dim,fragmentation,height,length,n_bifs,n_branch,n_stems,neuron_name,partition_asymmetry,pathDistance,pk_classic,soma_Surface,surface,volume,width,br1,br2,br3,br4,ds_layer,hemisphere,raw_layer,exact_layer,rough_layer,cell_type_1,cell_type_2,cell_type_2_curated,fastspike,SOM,PV,NPY,CR,5HT3,VIP,CCK,SOM_PV,label1
0,young,Markram,"No Diameter, 3D, Angles","[neocortex, somatosensory, layer 4]","[Large, basket, interneuron]",2005-12-31,"Dendrites, Soma, Axon",Control,60,16.0,None,13.0,None,240,Tissue shrinkage - corrected only in z-axis,Not reported,Neurolucida.asc,"Dendrites Complete, Axon Moderate",in vitro,Neurolucida,[11884355],rattus norvegicus,Y,Reported,parasagittal,80.00,936.38,rat,biocytin,Wistar,2006-08-01,True,True,81.47,23.0,0.86,259.82,0.41,807.95,1.04,4261.0,1115.42,21166.60,188.0,384.0,8.0,C010398B-I4,0.58,1217.48,1.81,936.38,25467.00,3117.46,955.12,neocortex,somatosensory,None,None,deep,None,layer 4,layer 4,layer 4,interneuron,"[Large, basket]",basket,False,False,False,False,False,False,False,False,unknown,cortex basket
1,young,Markram,"No Diameter, 3D, Angles","[neocortex, somatosensory, layer 4]","[Nest, basket, interneuron]",2005-12-31,"Dendrites, No Soma, Axon",Control,60,14.0,None,14.0,None,243,Tissue shrinkage - corrected only in z-axis,water or oil,Neurolucida.asc,"Dendrites Complete, Axon Moderate",in vitro,Neurolucida,"[11884355, 15946970]",None,Y,Reported,parasagittal,200.00,685.11,rat,biocytin,Han Wistar,2006-08-01,True,True,78.26,14.0,0.89,231.33,0.39,427.14,1.03,3120.0,321.13,14487.20,148.0,303.0,7.0,C010600A2,0.52,625.66,1.87,685.11,15609.30,1897.66,336.51,neocortex,somatosensory,None,None,deep,None,layer 4,layer 4,layer 4,interneuron,"[Nest, basket]",basket,False,False,False,False,False,False,False,False,unknown,cortex basket
2,young,Markram,"No Diameter, 3D, Angles","[neocortex, somatosensory, layer 4]","[Large, basket, interneuron]",2005-12-31,"Dendrites, No Soma, Axon",Control,60,14.0,None,14.0,None,244,Tissue shrinkage - corrected only in z-axis,water or oil,Neurolucida.asc,"Dendrites Complete, Axon Moderate",in vitro,Neurolucida,"[11884355, 15946970]",None,Y,Reported,parasagittal,200.00,904.86,rat,biocytin,Han Wistar,2006-08-01,True,True,83.31,26.0,0.86,317.07,0.37,477.72,1.04,9436.0,691.87,38659.60,356.0,720.0,8.0,C010600B1,0.51,1266.01,1.91,904.86,42711.10,5654.74,466.63,neocortex,somatosensory,None,None,deep,None,layer 4,layer 4,layer 4,interneuron,"[Large, basket]",basket,False,False,False,False,False,False,False,False,unknown,cortex basket
3,young,Markram,"No Diameter, 3D, Angles","[neocortex, somatosensory, layer 4]","[Large, basket, interneuron]",2005-12-31,"Dendrites, No Soma, Axon",Control,60,14.0,None,14.0,None,245,Tissue shrinkage - corrected only in z-axis,water or oil,Neurolucida.asc,"Dendrites Complete, Axon Moderate",in vitro,Neurolucida,"[11884355, 15946970]",None,Y,Reported,parasagittal,200.00,853.69,rat,biocytin,Han Wistar,2006-08-01,True,True,81.62,17.0,0.80,619.17,0.39,598.57,1.06,4794.0,832.38,25547.40,161.0,329.0,7.0,C010600B2,0.55,1147.81,1.86,853.69,28924.80,3831.27,480.27,neocortex,somatosensory,None,None,deep,None,layer 4,layer 4,layer 4,interneuron,"[Large, basket]",basket,False,False,False,False,False,False,False,False,unknown,cortex basket
4,young,Markram,"No Diameter, 3D, Angles","[neocortex, somatosensory, layer 2-3]","[Small, basket, interneuron]",2005-12-31,"Dendrites, No Soma, Axon",Control,60,14.0,None,14.0,None,246,Tissue shrinkage - corrected only in z-axis,water or oil,Neuroluc

In [7]:
neurons_all = pd.concat([principal_cells, interneurons], axis=0)

# delete bad neurons

In [15]:
neurons_all = neurons_all[~neurons_all.neuron_id.isin([247, 407])]  # no dendrites although it has dendrite in domains

In [17]:
neurons_all.label1.value_counts()

cortex pyramidal superficial    3691
cortex pyramidal deep           3466
None                            2523
hippocampus granule             1881
striatum medium spiny           1484
retina ganglion                 1297
hippocampus pyramidal           1169
cortex basket                    515
retina bipolar                   446
retina amacrine                  369
amygdala pyramidal               353
cortex Martinotti                282
olfactory bulb granule           282
Name: label1, dtype: int64

In [18]:
# the none's are neurons without a deep/superficial label

In [19]:
neurons_all[neurons_all.label1=='None']

,3D,5HT3,CCK,CR,NPY,PV,SOM,SOM_PV,VIP,age_classification,angles,archive,attributes,bif_ampl_local,br1,br2,br3,br4,brain_region,branch_Order,cell_type,cell_type_1,cell_type_2,cell_type_2_curated,contraction,deposition_date,depth,diameter,domain,ds_layer,eucDistance,exact_layer,experiment_condition,fastspike,fractal_Dim,fragmentation,height,hemisphere,label1,length,magnification,max_age,max_weight,min_age,min_weight,n_bifs,n_branch,n_stems,neuron_id,neuron_name,note,objective_type,original_format,partition_asymmetry,pathDistance,physical_Integrity,pk_classic,protocol,raw_layer,reconstruction_software,reference_pmid,rough_layer,scientific_name,shrinkage_corrected,shrinkage_reported,slicing_direction,slicing_thickness,soma_Surface,soma_surface,species,stain,strain,surface,upload_date,volume,width
451,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,young,True,Yuste,"Diameter, 3D, Angles",70.0300,neocortex,somatosensory,None,None,"[neocortex, somatosensory]",8.0,"[pyramidal, principal cell]",principal cell,[pyramidal],pyramidal,0.820000,2008-02-06,213.50,0.190000,"Dendrites, Soma, Axon",None,319.750,NaN,Control,NaN,1.04000,854.0,284.790,None,None,1727.25,60,17.0,None,13.0,None,24.0,52.0,4.0,2198,010920-slice2-cellB,,water,Neurolucida.dat,0.510000,526.680,"Dendrites Moderate, Axon Incomplete",1.650000,in vitro,None,Neurolucida,[14622584],NaN,,,Not reported,coronal,300.00,107.80,107.8,mouse,biocytin,C57BL/6,989.860,2008-07-15,69.5300,101.8200
455,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,young,True,Yuste,"Diameter, 3D, Angles",75.3000,neocortex,occipital,None,None,"[neocortex, occipital]",14.0,"[pyramidal, principal cell]",principal cell,[pyramidal],pyramidal,0.820000,2008-02-06,38.00,0.280000,"Dendrites, Soma, Axon",None,993.880,NaN,Control,NaN,1.08000,4385.0,1051.830,None,None,10082.10,None,49.0,None,13.0,None,73.0,153.0,7.0,2229,AM101-3-2,,Not reported,Neurolucida.dat,0.530000,1432.010,Dendrites & Axon Moderate,1.630000,in vitro,None,Neurolucida,[17135406],NaN,,,Not reported,coronal,350.00,703.06,703.06,mouse,biocytin,C57BL/6,9073.200,2008-07-15,758.9800,754.7900
457,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,young,True,Yuste,"Diameter, 3D, Angles",78.1300,neocortex,occipital,None,None,"[neocortex, occipital]",16.0,"[pyramidal, principal cell]",principal cell,[pyramidal],pyramidal,0.830000,2008-02-06,126.00,0.260000,"Dendrites, Soma, Axon",None,652.620,NaN,Control,NaN,1.08000,6614.0,654.680,None,None,12446.80,None,49.0,None,13.0,None,114.0,234.0,6.0,2231,AM107-4-2,,Not reported,Neurolucida.dat,0.590000,1265.330,Dendrites & Axon Moderate,1.920000,in vitro,None,Neurolucida,[17135406],NaN,,,Not reported,coronal,350.00,473.00,473.0,mouse,biocytin,C57BL/6,10260.200,2008-07-15,715.7600,655.8700
467,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,young,True,Yuste,"Diameter, 3D, Angles",65.3000,neocortex,occipital,None,None,"[neocortex, occipital]",9.0,"[pyramidal, principal cell]",principal cell,[pyramidal],pyramidal,0.830000,2008-02-06,34.50,0.270000,"Dendrites, Soma, Axon",None,404.490,NaN,Control,NaN,1.05000,2333.0,537.340,None,None,4395.64,None,49.0,None,13.0,None,39.0,85.0,7.0,2242,AM47-2-2,,Not reported,Neurolucida.dat,0.510000,537.380,Dendrites & Axon Moderate,1.640000,in vitro,None,Neurolucida,[17135406],NaN,,,Not reported,coronal,350.00,656.48,656.48,mouse,biocytin,C57BL/6,3950.540,2008-07-15,346.1000,364.1900
468,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,young,True,Yuste,"Diameter, 3D, Angles",57.7000,neocortex,occipital,None,None,"[neocortex, occipital]",6.0,"[pyramidal, principal cell]",principal cell,[pyramidal],pyramidal,0.800000,2008-02-06,38.00,0.260000,"Dendrites, Soma, Axon",None,592.630,NaN,Control,NaN,1.08000,1811.0,735.990,None,None,3616.35,None,49.0,None,13.0,None,27.0,63.0,9.0,2244,AM47-3-2,,Not reported,Neurolucida.dat,0.460000,752.210,Dendrites & Axon Moderate,1.670000,in vitro,None,Neurolucida,[17135406],NaN,,,Not reported,coronal,350.00,751.36,751.36,mouse,biocytin,C57BL/6,3342.730,2008-07-15,282.9500,272.5500
469,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [20]:
neurons_all.cell_type_2_curated.value_counts()

pyramidal       11202
granule          2163
medium spiny     1484
ganglion         1297
basket            515
bipolar           446
amacrine          369
Martinotti        282
Name: cell_type_2_curated, dtype: int64

In [21]:
neurons_all.to_pickle('../data/rodent_3d_dendrites_br-ct-filter-3_all_mainclasses.pkl')
